# <a id='toc1_'></a>[Using SD3.5 with AWS JumpStart](#toc0_)

This sample notebook shows you how to deploy Stable Diffusion 3.5 (SD3.5) Large from Stability AI as an endpoint on Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.


## <a id='toc1_1_'></a>[Prerequisites](#toc0_)

1. **Note**: Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.

1. Ensure that IAM role used has **AmazonSageMakerFullAccess**

1. To deploy the ML model successfully using the steps in this notebook, ensure that either:
    1. Your IAM role has the following three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. Or your AWS account has a subscription to [the Stable Diffusion 3.5 Large Jumpstart]. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package) 




## <a id='toc1_2_'></a>[Resources](#toc0_)


1. [Stability Stable Diffusion 3.5 Large documentation](https://platform.stability.ai/docs/api-reference#tag/Generate/paths/~1v2beta~1stable-image~1generate~1sd3/post)

2. [Documentation on real-time inference with Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).


## <a id='toc1_3_'></a>[Usage instructions](#toc0_)
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).


   
- [1. Subscribe to the SD3.5 Model Package](#toc3_)    
- [2: Create an endpoint and perform real-time inference](#toc4_)    
  - [A: Text to image](#toc4_1_)    
  - [B: Image to image](#toc4_2_)    
- [3: Delete the endpoint](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc3_'></a>[1. Subscribe to the Stable Diffusion 3.5 Large Model Package](#toc0_)

To subscribe to the Stable Diffusion 3.5 Large Model Package:
1. Open the Stable Diffusion 3.5 Large Model Package listing page: <<<   TODO >>>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization accept the EULA, pricing, and support terms.

In [ ]:
import sagemaker
from sagemaker import ModelPackage, get_execution_role


from PIL import Image
from typing import Union, Tuple
import io
import os
import base64
import boto3
import json

# <a id='toc4_'></a>[2: Create an endpoint and perform real-time inference](#toc0_)

In [ ]:
# Choose your endpoint name
from sagemaker.utils import name_from_base
endpoint_name=name_from_base('sd3-5-large-jumpstart') # change name as desired

Once you have subscribed to Stability Stable Diffusion 3.5 Large, get the Model Package ARN using the map below:


In [ ]:

model_package_map = {
    "us-east-1": 'arn:aws:sagemaker:us-east-1:188650660114:model-package/sd3-5-large',
    # "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad",
    # "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/sdxl-v1-0-8cc703e-43ceeb816ad635d18270e159eb5096ad"
}


region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")
package_arn = model_package_map[region]

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
# Verify the active account
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
print(f"Active account ID: {account_id}")

Create a deployable `ModelPackage`. For SD3.5, deploy onto a ml.p5.48xlarge instance. Specify it as `instance_type` below.


In [ ]:

model = ModelPackage(role=role,
                     model_package_arn=package_arn,
                     sagemaker_session=sagemaker_session)


# Deploy the ModelPackage. This will take 20-25 minutes to run

instance_type="ml.p5.48xlarge"
deployed_model = model.deploy(initial_instance_count=1,instance_type=instance_type,endpoint_name=endpoint_name)

We can invoke our deployed model to return model outputs. For the full list of request parameters, [see the Stability.ai API documentation.](https://api.stability.ai/docs#tag/v1generation)

## <a id='toc4_1_'></a>[A: Text to image](#toc0_)


In [ ]:
# Now we can invoke the model using boto
sm_runtime = boto3.client("sagemaker-runtime")

params = {
    "prompt": "a high-def photograph of a fisherman on the beach, caribean island, storm clouds",
    "seed": 101,
    "aspect_ratio": "21:9",
    "output_format": "jpeg",
}

payload = json.dumps(params).encode("utf-8")

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)

out = json.loads(response["Body"].read().decode("utf-8"))
try:
    base64_string = out["body"]["images"][0]
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    display(image)

except:
    print(out)


In [ ]:
out

In [ ]:

params = {
    "prompt": "a mural of a toucan in bright, fresh colors painted on the side of a house in brazil",
    "seed": 3,
    "output_format": "jpeg",
}
payload = json.dumps(params).encode("utf-8")

output = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)

Output images are included in the response body `images` as base64 encoded strings. Below is a helper function for accessing decoding these images:

In [ ]:
def decode_and_show(model_response: dict) -> None:
    """
    Decodes and displays an image from SD3.5 output

    Args:
        model_response (dict): The response object from the deployed model.

    Returns:
        None
    """
    out = json.loads(model_response["Body"].read().decode("utf-8"))
    base64_string = out["body"]["images"][0]
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    display(image)



In [ ]:
decode_and_show(output)

In [ ]:
params = {
    "prompt": "a selfie of a woman in a market in east aisa, holding a bag of groceries",
    "seed": 11111,
    "output_format": "jpeg",
}
payload = json.dumps(params).encode("utf-8")

output = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)
decode_and_show(output)


Let's try generating a snippet of text.

In [ ]:
text = "the word SD3.5 JUMPSTART written in neon lights"

params = {
    "prompt": text,
    "seed": 7,
    "output_format": "jpeg",
}
payload = json.dumps(params).encode("utf-8")

output = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)
decode_and_show(output)

### Aspect ratios
To control the size of a text-to-image request, pass in an aspect ratio from the list ` 16:9 1:1 21:9 2:3 3:2 4:5 5:4 9:16 9:21` as a string. The default aspect ratio is 1:1.

For text banners, use aspect ratios corresponding to long images. Likewise, for best results, specify approprate aspect ratios for portraits, landscapes, and square images.

Let's generate some unique text banners with the aspect ratio 21:9.

In [ ]:
text_prompt = "Create a bold and dynamic text design for the word 'ADHERENCE' with each letter filled with vibrant and high-fashion photography scenes. Incorporate a mix of models striking elegant poses, cameras flashing, and creative studio setups. Highlight the diversity of the modeling world with a variety of model expressions, runway moments, and behind-the-scenes shots. Use sleek, modern colors that reflect professionalism, creativity, and innovation, integrating camera lenses, softboxes, and fashion accessories within the letters to emphasize the photography and modeling theme. The overall design should feel high-end, artistic, and tailored for a professional audience."

params = {
    "prompt": text_prompt,
    "seed": 1000,
    "aspect_ratio": "21:9",
    "output_format": "jpeg",
}

payload = json.dumps(params).encode("utf-8")

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)

out = json.loads(response["Body"].read().decode("utf-8"))
try:
    base64_string = out["body"]["images"][0]
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    display(image)

except:
    print(out)

## <a id='toc4_2_'></a>[B: Image to image](#toc0_)

To perform inference that takes an image as input, you must set the parameter `mode` to `image-to-image`, and pass an input image into `init_image` as a base64-encoded string. 

Images can be jpeg, png or webp format, and each side must be between 64 and 1536 pixels.


Below is a helper function for converting images to base64-encoded strings:

In [ ]:
def encode_image(image_path: str, resize: bool = True, size: Tuple[int, int] = (1024, 1024)) -> Union[str, None]:
    """
    Encode an image as a base64 string, optionally resizing it to a supported resolution.

    Args:
        image_path (str): The path to the image file.
        resize (bool, optional): Whether to resize the image. Defaults to True.

    Returns:
        Union[str, None]: The encoded image as a string, or None if encoding failed.
    """
    assert os.path.exists(image_path)

    if resize:
        image = Image.open(image_path)
        image = image.resize(size)
        image.save("image_path_resized.png")
        image_path = "image_path_resized.png"
    image = Image.open(image_path)
    assert image.size == size
    with open(image_path, "rb") as image_file:
        img_byte_array = image_file.read()
        # Encode the byte array as a Base64 string
        try:
            base64_str = base64.b64encode(img_byte_array).decode("utf-8")
            return base64_str
        except Exception as e:
            print(f"Failed to encode image {image_path} as base64 string.")
            print(e)
            return None
    

Let's feed an image into the model as well as the prompt this time. We can set `strength` to weight the relative importance of the image versus the prompt. For the demo, we'll use a [picture of the cat, taken from Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Cat_August_2010-4.jpg), provided along with this notebook.

In [ ]:
! wget https://platform.stability.ai/Cat_August_2010-4.jpg

In [ ]:
# Here is the original image:
display(Image.open('Cat_August_2010-4.jpg'))

In [ ]:
cat_path = "Cat_August_2010-4.jpg"

size = (1536, 640)
cat_data = encode_image(cat_path, size=size)


params = {
    "prompt": "a child's painting of a cat, painted with bright paints",
    "seed": 123,
    "output_format": "jpeg",
    "mode": "image-to-image",
    "image": cat_data,
    "strength": 0.8,
}
payload = json.dumps(params).encode("utf-8")

output = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=payload,
)
decode_and_show(output)

# <a id='toc5_'></a>[3: Delete the endpoint](#toc0_)

When you've finished working, you can delete the endpoint to release the EC2 instance(s) associated with it, and stop billing.

Get your list of Sagemaker endpoints using the AWS Sagemaker CLI like this:

In [ ]:
!aws sagemaker list-endpoints

# Delete an endpoint

In [ ]:
deployed_model.sagemaker_session.delete_endpoint(endpoint_name)
# Rerun the aws cli command above to confirm that its gone.